## 基于K近邻(回归)（K nearest neighbour）

### k-近邻算法原理
- K近邻(回归)模型是无参数模型，只是借助K个最近训练样本的目标数值，对待测样本的回归值或分类进行决策,即根据样本的相似度预测回归值。既可处理分类问题，也可处理回归问题，其中分类和回归的主要区别在于最后做预测时的决策方式不同。KNN做回归预测时一般采用平均法，预测结果为最近的K个样本数据的平均值。
- 存在一个样本数据集合，也称作训练样本集，并且样本集中每个数据都存在标签，即我们知道样本集中每一数据 与所属分类的对应关系。输人没有标签的新数据后，将新数据的每个特征与样本集中数据对应的 特征进行比较，然后算法提取样本集中特征最相似数据（最近邻）的分类标签。一般来说，我们 只选择样本数据集中前K个最相似的数据，这就是K-近邻算法中K的出处,通常K是不大于20的整数。 最后 ，选择K个最相似数据中出现次数最多的分类，作为新数据的分类。
- 欧几里得距离(Euclidean Distance)
欧氏距离是最常见的距离度量，衡量的是多维空间中各个点之间的绝对距离。公式如下：
$ \mathrm dist(\boxed{\boldsymbol {X,Y}})=\sqrt{\sum \limits _{i = 1}^n {(x_i - y_i)^2} } $
- K值选择
K的取值比较重要,答案是通过交叉验证,从选取一个较小的K值开始，不断增加K的值，然后计算验证集合的方差，最终找到一个比较合适的K值。
- KNN算法优点
  * 简单易用，相比其他算法，KNN算是比较简洁明了的算法。即使没有很高的数学基础也能搞清楚它的原理。
  * 模型训练时间应该快，但我的运行时间较长，KNN算法是惰性的,有监督
  * 预测效果好
  * 对异常值不敏感
- KNN算法缺点
  * 对内存要求较高，因为该算法存储了所有训练数据
  * 预测阶段可能很慢
  * 对不相关的功能和数据规模敏感

### （KNeighborsClassifier/KNeighborsRegressor）
适用数据范围：数值型和标称型。
- 衡量样本待测样本回归值的不同方式:
  * 对K个近邻目标数值使用普通的算术平均算法
  * 对K个近邻目标数值考虑距离的差异进行加权平均
- 选择合适的特征,建立模型训练
- fit(X,y)，在（X，y）上运行记分函数并得到适当的特征。
- fit_transform(X[, y])，拟合数据，然后转换数据。

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

X_train = pd.read_csv("KNN_X_train.csv")
Y_train = pd.read_csv("KNN_Y_train.csv")
X_validation = pd.read_csv("KNN_X_validation.csv")
Y_validation = pd.read_csv("KNN_Y_validation.csv")
X_test = pd.read_csv("KNN_X_test.csv")

In [16]:
print(X_validation.shape)
print(Y_validation.shape)

(990360, 27)
(990360, 1)


In [8]:
knn_features = ['item_cnt', 'item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1',
                'item_cnt_shifted2', 'shop_mean', 'shop_item_mean', 
                'item_trend', 'mean_item_cnt']
 
 
# Subsample train set (using the whole data was taking too long).
X_train_sampled = X_train
Y_train_sampled = Y_train

##X_train_sampled = X_train[:100000]
##Y_train_sampled = Y_train[:100000]
##
 
knn_train = X_train_sampled[knn_features]
knn_val = X_validation[knn_features]
knn_test = X_test[knn_features]
 
 #可以将任意数值归一化处理到一定区间
knn_scaler = MinMaxScaler()
knn_scaler.fit(knn_train)
knn_train = knn_scaler.transform(knn_train)
knn_val = knn_scaler.transform(knn_val)
knn_test = knn_scaler.transform(knn_test)
 
knn_model = KNeighborsRegressor(n_neighbors=9, leaf_size=13, n_jobs=-1)
knn_model.fit(knn_train, Y_train_sampled)


KNeighborsRegressor(leaf_size=13, n_jobs=-1, n_neighbors=9)

In [20]:
print(Y_train_sampled.shape)
print(Y_validation.shape)
print(knn_val.shape)

(4951800, 1)
(990360, 1)
(990360, 9)


In [9]:
#代码错误，仅以教训
from sklearn.metrics import mean_squared_error
knn_train_pred=knn_model.predict(knn_train)
knn_val_pred=knn_model.predict(knn_val)
knn_test_pred=knn_model.predict(knn_test)

print("Train rmse:",np.sqrt(mean_squared_error(Y_train_sampled,knn_train_pred)))
print("Validation rmse:",np.sqrt(mean_squared_error(Y_validation,knn_val_pred)))

Train rmse: 2.005091380122831


ValueError: Found input variables with inconsistent numbers of samples: [990360, 4951800]

In [21]:
from sklearn.metrics import mean_squared_error
knn_train_pred=knn_model.predict(knn_train)
knn_val_pred=knn_model.predict(knn_val)
knn_test_pred=knn_model.predict(knn_test)

In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4951800 entries, 0 to 4951799
Data columns (total 27 columns):
 #   Column               Dtype  
---  ------               -----  
 0   shop_id              int64  
 1   item_id              float64
 2   item_price           float64
 3   mean_item_price      float64
 4   item_cnt             float64
 5   mean_item_cnt        float64
 6   transactions         float64
 7   year                 int64  
 8   month                int64  
 9   item_price_unit      float64
 10  hist_min_item_price  float64
 11  hist_max_item_price  float64
 12  price_increase       float64
 13  price_decrease       float64
 14  item_cnt_min         float64
 15  item_cnt_max         float64
 16  item_cnt_mean        float64
 17  item_cnt_std         float64
 18  item_cnt_shifted1    float64
 19  item_cnt_shifted2    float64
 20  item_cnt_shifted3    float64
 21  item_trend           float64
 22  shop_mean            float64
 23  item_mean            float64
 24

In [38]:
print("Train rmse:",np.sqrt(mean_squared_error(Y_train_sampled,knn_train_pred)))
print("Validation rmse:",np.sqrt(mean_squared_error(Y_validation,knn_val_pred)))

Train rmse: 2.005091380122831
Validation rmse: 2.644335562537569


In [33]:
print(Y_validation.shape)
print(knn_train_pred.shape)
print(knn_test_pred.shape)
test=pd.read_csv("test.csv")
print(test.shape)
print(test['ID'].shape)
print(test.index)

(990360, 1)
(4951800, 1)
(214200, 1)
(214200, 3)
(214200,)
RangeIndex(start=0, stop=214200, step=1)


In [36]:
# 使用sklearn调用衡量线性回归的MSE 、 RMSE、 MAE、r2
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
print("mean_absolute_error:", mean_absolute_error(Y_validation,knn_val_pred))
print("mean_squared_error:", mean_squared_error(Y_validation,knn_val_pred))
print("rmse:", sqrt(mean_squared_error(Y_validation,knn_val_pred)))
print("r2 score:", r2_score(Y_validation,knn_val_pred))


mean_absolute_error: 0.31104514183394605
mean_squared_error: 6.99251056730088
rmse: 2.644335562537569
r2 score: 0.3718881342620014


In [34]:
#保存预测值
test=pd.read_csv("test.csv")
prediction_df = pd.DataFrame(test['ID'], columns=['ID'])
prediction_df['item_cnt_month'] = knn_test_pred

prediction_df.to_csv('KNN_test_submission.csv', index=False)


RangeIndex(start=0, stop=214200, step=1)
